In [4]:
import os
import bs4
import re
import pandas as pd
import numpy as np
import datetime
from collections import Counter

files = os.listdir('Ikon/tmp')

In [8]:
def add_fields(sp, tag, cls):
    res = sp.findAll(tag, {'class' : cls})
    if len(res) > 0:
        return res[0].text.strip()
    else:
        return None
    
 
    
all_exhibitions = []   
all_artists = []
all_galleries = []

names_dates = {}

already_processed = set()

for fn in files:
    
    if 2 == 2:
    #if 'IkOn2015.01.26.Mon-2015.02.01.Sun' in fn:
        
        with open('./Ikon/tmp/' + fn, encoding="utf-8") as myfile:
            html = myfile.read()
            soup = bs4.BeautifulSoup(html,"lxml")
            exbhis = soup.findAll('div', {'class' : "row"})
            
            for ex in exbhis:
                
                exhibition = {}
        
                year_m  = add_fields(ex, 'div', 'year-month')
                
                if year_m:
                
                    title = add_fields(ex, 'a',   'title')
                
                    if title not in already_processed:

                        already_processed.add(title)

                        day     = add_fields(ex, 'div', 'day')
                        hour    = add_fields(ex, 'div', 'hour')
                        year_m  = ''.join([s for s in year_m if (s == '.' or s.isdigit())])
                        date    = datetime.datetime.strptime(year_m + ' ' + day + ' ' + hour, '%Y.%m. %d %H:%M')
                        gallery = add_fields(ex, 'a',   'gallery')

                        exhibition['date']    = date
                        exhibition['title']   = title
                        exhibition['gallery'] = gallery
                        artists               = ex.findAll('', {'class' : 'artists'}) 
                        exhibition['artists'] =  '; '.join([a.text.replace(';' , ' ') for a in artists])
                        all_exhibitions.append(exhibition)

                        
                        

                        all_artists += [a.text for a in artists]
                        
                        for a in artists:
                            if a.text not in names_dates:
                                names_dates[a.text] = []
                            names_dates[a.text].append(date)
                        
                        all_galleries.append(gallery)
                    

df = pd.DataFrame(all_exhibitions)                    

In [9]:
df

,date,title,gallery,artists
0,1999-11-12 18:00:00,"Szentendrei művészet a 70-es, 80-as években - ...","Városi Művészeti Múzeum Képtára, Győr","Vajda Lajos Stúdió,; Ef Zámbó István"
1,1999-12-07 18:00:00,Actionman,Fiatal Képzőművészek Stúdiója - Stúdió Galéria,"Kupcsik Adrián,; Bencsik Barnabás"
2,2000-01-01 01:00:00,,MAMŰ Galéria,"Korok,; -,; -; L,; -,; -; gthg; mjbkjhb; Ágoth..."
3,2000-01-01 01:00:00,blair witch-projekt,Liget Galéria,Szabó Ágnes
4,2000-01-01 01:00:00,GRAFIKAI BEMUTATÓ,Képíró Galéria,"Bálint Endre,; Ország Lili,; Barta István,; Sz..."
...,...,...,...,...
13260,2020-01-04 11:30:00,Hangover Reading Club #4 - Let's go with the f...,Trafó Galéria,.; Rajk László; Rajk László; Művészeti foglalk...
13261,2019-12-30 11:00:00,Gyermeki varázskör Jankovics Marcell kiállításán,Műcsarnok,Művészeti foglalkozás
13262,2020-01-03 19:00:00,BONNE ANNÉE BONNE SANTÉ! NAGY FORMÁTUMOK KIS F...,Három Hét Galéria,"Szöllősi-Nagy - Nemes Gyűjtemény,; Szöllősi-Na..."
13263,2019-12-30 16:00:00,Gryllus Dániel | Pál apostol - Dalok Pál level...,Műcsarnok,"Közreműködik: Szirtes Edina Mókus,; Jankovics ..."


In [10]:
fout = open('Ikon/extracted/names_min_max_date.dat', 'w')
for n, dates in names_dates.items():
    if len(n) > 0 and len(dates) > 0:
        fout.write(n.encode('utf-8').strip() + '\t' + str(min(dates)) + '\t' + str(max(dates)) + '\n' )
fout.close()

TypeError: can't concat str to bytes

In [11]:
folderout = 'extracted'
if not os.path.exists(folderout):
    os.makedirs(folderout)
    
df = df.drop_duplicates()    
    
len(df)

13265

In [12]:
df.to_csv('Ikon/extracted/ikon_exhibitions_raw.csv', sep = '\t',encoding='utf-8')

fout = open('Ikon/extracted/all_artists_raw.dat', 'w')
all_artists_s = sorted(list(set(all_artists)))
for a in all_artists_s:
    fout.write(a.encode('utf-8').strip()+'\n')
fout.close()


fout = open('Ikon/extracted/all_galleries_raw.dat', 'w')
all_galleries_s = sorted(list(set(all_galleries)))
for a in all_galleries_s:
    fout.write(a.encode('utf-8').strip()+'\n')
fout.close()

TypeError: can't concat str to bytes

In [13]:
artists_cnt = Counter(all_artists)
artists_cnt

Counter({'Vajda Lajos Stúdió,': 3,
         'Ef Zámbó István': 1,
         'Kupcsik Adrián,': 46,
         'Bencsik Barnabás': 41,
         'Korok,': 1,
         '-,': 2997,
         '-': 2027,
         'L,': 2,
         'gthg': 1,
         'mjbkjhb': 1,
         'Ágotha Margit,': 9,
         'Borbás Márton,': 5,
         'Czeglédi Adél,': 3,
         'Demeter István,': 11,
         'Dömény Boriska,': 3,
         'Fazekas Magdolna,': 21,
         'Gránitz Tamás,': 3,
         'borgó': 1,
         'Szabó Ágnes': 6,
         'Bálint Endre,': 24,
         'Ország Lili,': 25,
         'Barta István,': 2,
         'Szász Endre,': 2,
         'Borsos Miklós,': 14,
         'Ruttkay Sándor,': 8,
         'Gross Arnold,': 10,
         'Molnár Gabriella,': 2,
         'Korniss Dezső,': 24,
         'Kass János,': 11,
         'Victor Vasarely,': 5,
         'Egyed László,': 11,
         'Ozoli Iza,': 12,
         'FAA,': 1,
         'Kassák Lajos': 4,
         'Expedíció a jövőbe': 2,
         

In [14]:
fout = open('Ikon/extracted/all_artists_cnt.dat', 'w')
artists_cnt = Counter(all_artists)
for (a, c) in dict(artists_cnt).items():
    fout.write(a.encode('utf-8').strip() + '\t' + str(c) +'\n')
fout.close()

fout = open('Ikon/extracted/all_galleries_cnt.dat', 'w')
galleries_cnt = Counter(all_galleries)
for a, c in dict(galleries_cnt).items():
    fout.write(a.encode('utf-8').strip() + '\t' + str(c) +'\n')
fout.close()

TypeError: can't concat str to bytes